# Prompt selection and testing

This notebook chooses the most appropriate prompt and prompt structure for the OCR correction. 

In [21]:
#import config  # Import your config.py file this contains you openai api key
import pandas as pd
import numpy as np
import os
from llm_comparison_toolkit import RateLimiter, get_response_openai, get_response_anthropic,  create_config_dict_func, compare_request_configurations, generate_model_configs
from evaluate import load
from evaluation_funcs import evaluate_correction_performance, evaluate_correction_performance_folders, get_metric_error_reduction
import seaborn as sns
import matplotlib.pyplot as plt
from helper_functions import files_to_df_func, evaluate_ner, calculate_entity_similarity, repeat_prompt_experiment
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
from openai import OpenAI

client = OpenAI()


dev_data_folder = 'data/dev_data'
dev_transcripts = os.path.join(dev_data_folder, 'dev_data_transcript')
dev_raw_ocr_folder =  os.path.join(dev_data_folder,'dev_raw_ocr' )
dev_system_message_folder = os.path.join(dev_data_folder,'dev_system_message_variants' )

#load the dev and test sets for prompt development and selection
dev_data_df = pd.read_csv(os.path.join(dev_data_folder,'dev_data_raw.csv'))


#for saving data to be used in the analysis
if not os.path.exists('data/analysis'):
    os.makedirs('data/analysis')

# Demonstrating context leveraging

The below texts can be used to demonstrate that LMs use context leveraging and that they can self-contextualise from the task itself. The words in the parenthesis are optional contxt

"""Please correct the below sentences containing OCR errors

My dog has no ***

*** did *** **** cross ***"""

In [2]:



response = get_response_openai(
    """Please correct the below sentences containing OCR errors

My dog has no ***

*** did *** **** cross ***""", 
"", RateLimiter(40000), 
engine="gpt-4-turbo-preview", 
max_tokens = 500)

2024-04-16 06:58:02 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [210]:

dog_nose = """My dog *** *** nose. 
How *** he ***? """

dog_nose = """My dog has no  ***"""

chicken_cross = "*** did *** **** cross ***"

chicken_cross = "why did the **** *** ***"


prompt_dog =f"""Please correct the below sentences containing OCR errors

{dog_nose}

"""

prompt_dog_joke =f"""Please correct the below sentences containing OCR errors, the sentence is part of a popular joke

{dog_nose}

"""

check_prompt_dog = """The below text surrounded by triple colons is the response of a language model to a task, did it correctly identify the sentence 'my dog has no nose', Answer 1 or 0
:::
{answer}
:::

"""

prompt_chicken =f"""Please correct the below sentences containing OCR errors

{chicken_cross}

"""

prompt_chicken_joke =f"""Please correct the below sentences containing OCR errors, the sentence is part of a popular joke

{chicken_cross}

"""

check_prompt_chicken = """The below text surrounded by triple colons is the response of a language model to a task, did it correctly identify the sentence 'why did the chicken cross the road', Answer 1 or 0
:::
{answer}
:::

"""


prompt_combined =f"""Please correct the below sentences containing OCR errors

{dog_nose}

{chicken_cross}
"""

prompt_combined_joke =f"""Please correct the below sentences containing OCR errors, the sentences are part of a popular jokes

{dog_nose}

{chicken_cross}
"""

check_combined = """The below text surrounded by triple colons is the response of a language model to a task, did it correctly identify both the sentences 'my dog has no nose' and 'why did the chicken cross the road', Answer 1 if both identified or 0 otherwise
:::
{answer}
:::

"""


In [232]:
context_check = pd.DataFrame([
             {'prompt': prompt_dog, 'check':check_prompt_dog, 'context':'no', 'joke':'dog' },
             {'prompt': prompt_dog_joke, 'check':check_prompt_dog, 'context':'yes', 'joke':'dog' },
             {'prompt': prompt_chicken, 'check':check_prompt_chicken, 'context':'no', 'joke':'chicken' },
             {'prompt': prompt_chicken_joke, 'check':check_prompt_chicken, 'context':'yes', 'joke':'chicken' },
             {'prompt': prompt_combined, 'check':check_combined, 'context':'no', 'joke':'combined' },
             {'prompt': prompt_combined_joke, 'check':check_combined, 'context':'yes', 'joke':'combined' }
            ])


context_results =[]
for index, row in context_check.iterrows():

    result = repeat_prompt_experiment(row['prompt'] , row['check'] , client, repititions = 100)

    result['context'] = row['context']
    result['joke'] = row['joke']

    context_results.append(result)


context_results = pd.concat(context_results, ignore_index=True)

#context_results.to_csv("does_LM_use_context_results.csv")

2024-04-16 10:35:25 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:26 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:27 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:27 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:28 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:29 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:30 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:31 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-16 10:35:32 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 